In [1]:
# Client configuration for Azure OpenAI and Foundry
import asyncio

from agent_framework.azure import AzureAIAgentClient, AzureOpenAIChatClient
from agent_framework.observability import setup_observability 
from azure.identity import DefaultAzureCredential
from azure.identity.aio import DefaultAzureCredential as AsyncDefaultAzureCredential
from openai import AzureOpenAI
from dotenv import load_dotenv
import config

load_dotenv()
setup_observability()

credential = DefaultAzureCredential()
async_credential = AsyncDefaultAzureCredential()    
openai_client = AzureOpenAI(
    azure_endpoint=config.azure_openai_endpoint,
    azure_ad_token_provider=lambda: credential.get_token("https://cognitiveservices.azure.com/.default").token,
    api_version="2024-02-01"
)

chat_client = AzureOpenAIChatClient(
    credential=credential,
    deployment_name="gpt-5-chat",
    endpoint=config.azure_openai_endpoint
)

agent_client = AzureAIAgentClient(
    project_endpoint=config.foundry_project_endpoint,
    model_deployment_name="gpt-5-chat",
    async_credential=async_credential,
)


In [2]:
# Functions for semantic similarity evaluation 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict, Any 
from main import generate_embedding


def calculate_semantic_similarity(text1: str, text2: str) -> float:
    """Calculate semantic similarity between two texts using cosine similarity."""
    emb1 = generate_embedding(openai_client, text1)
    emb2 = generate_embedding(openai_client, text2)
    
    if not emb1 or not emb2:
        return 0.0

    # Convert to numpy arrays and reshape for cosine_similarity
    emb1 = np.array(emb1).reshape(1, -1)
    emb2 = np.array(emb2).reshape(1, -1)
    
    similarity = cosine_similarity(emb1, emb2)[0][0]
    return float(similarity)

def evaluate_similarity_score(similarity_score: float) -> str:
    """Evaluate similarity score and return quality rating."""
    if similarity_score >= 0.85:
        return "Excellent"
    elif similarity_score >= 0.75:
        return "Good"
    elif similarity_score >= 0.65:
        return "Satisfactory"
    else:
        return "Needs Improvement"


In [ ]:
# Define test questions and reference answers 
test_cases = [
    {
        "category": "Company Information",
        "query": "When was Meridian founded?",
        "reference_answer": "Meridian Strategic Consulting was founded in 2018"
    },
    {
        "category": "Service Offerings",
        "query": "What specific AI and automation services does Meridian offer, and what kind of ROI can clients expect?",
        "reference_answer": """Meridian offers comprehensive AI & Automation Implementation services including AI opportunity assessment, solution design and architecture, implementation and testing, and change management support. These engagements typically last 16-28 weeks with teams of 5-10 consultants. Clients can expect a typical ROI of 400-600%, making this one of our highest-value service offerings. Dr. Amanda Foster leads this practice with expertise in machine learning, having 12 patents in ML and automation, and the team has delivered $500M+ in client value through AI implementations."""
    },
    {
        "category": "Expert Identification", 
        "query": "I need a consultant with AI/ML expertise and healthcare industry experience for a 6-month project. Who would be the best match?",
        "reference_answer": """Dr. Amanda Foster would be the ideal match for this project. She is a Senior Partner with expertise in AI/ML and leads both the Digital Transformation practice and Healthcare vertical. She has a PhD in Computer Science from Carnegie Mellon, holds 12 patents in machine learning and automation, and has led AI implementations generating $500M+ in client value. She has specific healthcare experience with projects including AI-powered diagnostic tools, predictive analytics for patient outcomes, and clinical decision support systems. However, she is currently available starting February 2025, so timeline coordination would be needed."""
    }
]

📋 Loaded 3 test cases for evaluation


In [ ]:
from main import  show_agent_answer
from main import MeridianKnowledgeBaseAgent

agent_instructions = """
You are a helpful AI assistant. 

You will receive a question and exerpts of documents returned from a search service in the following format
```
Question: {question}
Exerpts: {context}
```

Keep thinking untill you have a complete answer. If the question contains multiple parts, split the query into parts and be sure to answer all parts of the question.

Use ONLY the provided knowledge base excerpts to answer the user's question as accurately as possible. 
Do not make assumptions or add information not contained in the excerpts.
If you cannot answer (part of the question) state which part of the question you are unable to answer.
"""

agent = MeridianKnowledgeBaseAgent(agent_instructions)

for test_case in test_cases:
    test_query = test_case["query"]
    reference_answer = test_case["reference_answer"]
    agent_response = await agent.ask(test_query)
    show_agent_answer(test_query, agent_response, reference_answer)

    print(f"🧪 Similarity Score: {calculate_semantic_similarity(agent_response, reference_answer)}\n")

🔍 Query:
 When was Meridian founded?

📝 Agent Response:
 Meridian Strategic Consulting was founded in 2018.

📚 Reference Answer:
 Meridian Strategic Consulting was founded in 2018

🧪 Similarity Score: 0.9658232205047548

🔍 Query:
 What specific AI and automation services does Meridian offer, and what kind of
ROI can clients expect?

📝 Agent Response:
 Meridian offers a range of AI and automation services designed to drive
efficiency, innovation, and measurable business outcomes.    **Specific AI and
Automation Services:**   - **AI Opportunity Assessment** – Identify high-impact
areas where AI can be deployed.   - **Solution Design and Architecture** –
Develop tailored AI-powered solutions and automation platforms.   -
**Implementation and Testing** – Deploy enterprise AI and automation systems to
ensure readiness and scalability.   - **Change Management and Training** – Equip
client teams to adopt and leverage AI solutions effectively.   - **Predictive
Models and Analytics Development*